In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change this to point to wherever you've put the data
data_dir = "F:\\GoogleDrive\\ds_ml\\RIT\\2022-01 DSCI.644\\project\\phase2\\"
data_path = data_dir + "project3-authors.csv"
df_raw = pd.read_csv(data_path)
# df_raw.shape   # (17317, 9)
# df_raw.columns # ['Name', 'CommitId', 'RefactoringType', 'FilePath', 'Class',
#                   'RefactoringDetail', 'Message', 'AuthorName', 'AuthorEmail']
# df_raw['Name'].unique()  # ['hibernate$hibernate-validator', 'eclipse$bpmn2-modeler',
#                             'adangel$pmd']
df_raw.head()

,Name,CommitId,RefactoringType,FilePath,Class,RefactoringDetail,Message,AuthorName,AuthorEmail
0,hibernate$hibernate-validator,a3af1683e63c9c800f486552e7e5002c24b6712b,Move Source Folder,src,NaN,Move Source Folder\tsrc to hibernate-validator...,Moved the legacy validator code into hibernate...,Hardy Ferentschik,hibernate@ferentschik.de
1,hibernate$hibernate-validator,6d6c3deaf0b499e06077383d8d3add43f30ba099,Move Source Folder,hibernate-validator-legacy/src,NaN,Move Source Folder\thibernate-validator-legacy...,Refactored build so that compile and test can ...,Hardy Ferentschik,hibernate@ferentschik.de
2,hibernate$hibernate-validator,6d6c3deaf0b499e06077383d8d3add43f30ba099,Move Source Folder,hibernate-validator-legacy/src/test,NaN,Move Source Folder\thibernate-validator-legacy...,Refactored build so that compile and test can ...,Hardy Ferentschik,hibernate@ferentschik.de
3,hibernate$hibernate-validator,25350de39ef9484997999f6e6d8f00abc5d51546,Extract Variable,hibernate-validator/src/main/java/org/hibernat...,org.hibernate.validation.engine.ValidatorImpl,Extract Variable\tleafBeanInstance : Object in...,BVAL-37 ConstraintDescriptor.getLeafValue()\n\...,Emmanuel Bernard,emmanuel@hibernate.org
4,hibernate$hibernate-validator,25350de39ef9484997999f6e6d8f00abc5d51546,Rename Method,hibernate-validator/src/main/java/org/hibernat...,org.hibernate.validation.engine.ValidatorImpl,Rename Method\tpublic getBeanConstraints() : E...,BVAL-37 ConstraintDescriptor.getLeafValue()\n\...,Emmanuel Bernard,emmanuel@hibernate.org


In [13]:
# filter for just the adangel$pmd rows
df_filtered = df_raw[df_raw['Name'] == "adangel$pmd"]
# df_filtered.shape  # (8495, 9) same row count as reported in phase 1 report, check
df_filtered.tail()

,Name,CommitId,RefactoringType,FilePath,Class,RefactoringDetail,Message,AuthorName,AuthorEmail
17312,adangel$pmd,a5f131cc4cfe59a7da3cd220353c402d3c612d76,Rename Method,pmd-ui/src/main/java/net/sourceforge/pmd/util/...,net.sourceforge.pmd.util.fxdesigner.popups.Eve...,Rename Method\tprivate bindPopupToController()...,Fix caching logic\n,Clément Fournier,clement.fournier76@gmail.com
17313,adangel$pmd,5a2202319f90080e9980735a37e4eb8e948d2f0f,Extract Method,pmd-ui/src/main/java/net/sourceforge/pmd/util/...,net.sourceforge.pmd.util.fxdesigner.MainDesign...,Extract Method\tpublic onNodeItemSelected(sele...,Highlight name declarations when selecting fro...,Clément Fournier,clement.fournier76@gmail.com
17314,adangel$pmd,5a2202319f90080e9980735a37e4eb8e948d2f0f,Rename Parameter,pmd-ui/src/main/java/net/sourceforge/pmd/util/...,net.sourceforge.pmd.util.fxdesigner.NodeInfoPa...,Rename Parameter\tfocusScopeView : boolean to ...,Highlight name declarations when selecting fro...,Clément Fournier,clement.fournier76@gmail.com
17315,adangel$pmd,d43e65761ba7a58025977e9e8ed414f05e3058ce,Rename Class,pmd-core/src/test/java/net/sourceforge/pmd/lan...,net.sourceforge.pmd.lang.ast.xpath.AttributeAx...,Rename Class\tnet.sourceforge.pmd.lang.ast.xpa...,[core] Undo changes to AttributeAxisIterator /...,Andreas Dangel,andreas.dangel@adangel.org
17316,adangel$pmd,d43e65761ba7a58025977e9e8ed414f05e3058ce,Rename Method,pmd-core/src/test/java/net/sourceforge/pmd/lan...,net.sourceforge.pmd.lang.ast.xpath.AttributeAx...,Rename Method\tpublic testAttributeAxisIterato...,[core] Undo changes to AttributeAxisIterator /...,Andreas Dangel,andreas.dangel@adangel.org


# Feature Creation

## Potential Features

### Assumptions

Uniformative columns:

+ CommitId
+ Email

### Exploratory Questions

How many unique types of each variable? How are these variables distributed?

+ RefactoringTypes
+ FilePath - How do we build a logical feature from this?
  - vector representation: cosine similarity should be higher among paths that share more path elements
  - e.g. if path 1 = a/b/c/d, path 2 = a/b/c/e, path 3 a/b/m/n then path 1 would have a higher cosine similarity to path 2 than to path 3.
  

In [7]:
# how are refactoring types distributed
refactor_types = df_filtered['RefactoringType'].value_counts()
# refactor_types.shape  # 27 types
refactor_types

Move Class                         1369
Rename Method                       967
Extract Method                      703
Rename Variable                     698
Rename Attribute                    688
Pull Up Method                      546
Move Method                         499
Rename Class                        467
Rename Parameter                    413
Move Attribute                      382
Extract Variable                    279
Push Down Method                    225
Extract And Move Method             195
Pull Up Attribute                   150
Extract Superclass                  122
Move And Rename Class               122
Inline Variable                     118
Inline Method                       103
Extract Class                        96
Change Package                       70
Parameterize Variable                65
Move Source Folder                   56
Push Down Attribute                  55
Replace Variable With Attribute      42
Extract Interface                    39


In [9]:
# how are file path distributed
file_types = df_filtered['FilePath'].value_counts()
# file_types.shape  # 3139 types
file_types.head(10)

pmd-eclipse-plugin/plugins/net.sourceforge.pmd.eclipse.plugin/src/net/sourceforge/pmd/eclipse/ui/nls/StringKeys.java                     193
pmd-apex/src/main/java/net/sourceforge/pmd/lang/apex/ast/AbstractApexNode.java                                                            97
pmd/src/net/sourceforge/pmd/PMD.java                                                                                                      77
pmd/src/net/sourceforge/pmd/ast/JavaParser.java                                                                                           75
pmd/src/main/java/net/sourceforge/pmd/lang/plsql/ast/PLSQLParser.java                                                                     75
pmd-java/src/main/java/net/sourceforge/pmd/lang/java/typeresolution/ClassTypeResolver.java                                                61
pmd/src/net/sourceforge/pmd/swingui/PMDViewer.java                                                                                        58
pmd-java/src/

In [12]:
# how are file path distributed
authors = df_filtered['AuthorName'].value_counts()
# authors.shape  # 77 authors, will want to bucket e.g. group1, >=1000 commits, group2, >200 but <1000
authors.head(10) # order of magnitude drop over the first 10

Tom Copeland        1624
Ryan Gustafson      1157
Brian Remedios      1094
Andreas Dangel       886
Clément Fournier     700
oowekyala            549
Xavier Le Vourch     323
Philippe Herlin      308
Don Leckie           234
Freddy Mallet        196
Name: AuthorName, dtype: int64